# Lamprey Transcriptome Analysis

```
Camille Scott [camille dot scott dot w @gmail.com] [@camille_codon]

camillescott.github.io

Lab for Data Intensive Biology (DIB)
University of California Davis
```

In [15]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


In [2]:
%run -i common.ipy

** Using data resources found in resources.json
** Using config found in config.json


In [3]:
store = pd.HDFStore(wdir('{}.store.h5'.format(prefix)), complib='zlib', complevel=5)

In [4]:
import atexit
def exit_func():
    dump_results()
    store.close()
atexit.register(exit_func)

<function __main__.exit_func>

In [7]:
orthologies = store['lamp10_ortho']

In [9]:
best_hits = store['lamp10_best_hits']

In [25]:
mm_ortho = orthologies['lamp10.fasta.x.musMus.pep.fa.db.tsv']
mm_bh = best_hits['lamp10.fasta.x.musMus.pep.fa.db.tsv']

In [59]:
def movingaverage(interval, window_size):
    window = numpy.ones(int(window_size))/float(window_size)
    return numpy.convolve(interval, window, 'same')

In [90]:
def get_estimator(measure, steps, ortho_df, bh_df):

    # For some bitscore X,
    # P(X|ortho) = P that an orthology has a best hit having bitscore X
    # P(X) = P of best hit having a bitscore of X
    # P(ortho) P of best hit being an orthology

    # P(ortho|X) = P(ortho)*P(X|ortho) / P(X)
    
    # Using theory of pseudocounts, we add 1 to each observation to allow for cases where
    # we fail to observe low-probability events

    
    end = bh_df[measure].max()
    step = float(end) / steps
    
    N_ortho = float(len(ortho_df.dropna()))
    N_bh = float(len(bh_df.dropna()))
    P_ortho = N_ortho / N_bh
    
    P_X_ortho = []
    for n in np.arange(0.0, end, step):
        n = float(len(ortho_df.query('{0} <= {1}_x < {2}'.format(n, measure, n + step)))) + 1.0
        P_X_ortho.append(n / N_ortho)
    #P_X_ortho = movingaverage(P_X_ortho, 3)
    
    P_X = []
    for n in np.arange(0.0, end, step):
        n = float(len(bh_df.query('{0} <= {1} < {2}'.format(n, measure, n + step)))) + 1.0
        P_X.append(n / N_bh)
    #P_X = movingaverage(P_X, 3)
    
    def estimator(val):
        p_xo = P_X_ortho[int(val) / int(step)]
        p_x = P_X[int(val) / int(step)]
        
        return (P_ortho * p_xo) / p_x
    
    return estimator

In [85]:
def get_actual(measure, steps, ortho_df, bh_df):

    end = bh_df[measure].max()
    step = float(end) / steps

    P_actual = []

    for n in np.arange(0.0, end, step):
        n_ortho = float(len(ortho_df.query('{0} <= {1}_x < {2}'.format(n, measure, n + step))))
        n_hit = float(len(bh_df.query('{0} <= {1} < {2}'.format(n, measure, n + step))))

        try:
            p = n_ortho / n_hit
        except ZeroDivisionError:
            p = '?'

        P_actual.append(p)
    
    def actual(val):
        return P_actual[int(val) / int(step)]
    
    return actual

In [86]:
est = get_estimator('bitscore', 25, mm_ortho, mm_bh)

In [82]:
act = get_actual('bitscore', 25, mm_ortho, mm_bh)

In [87]:
for n in np.arange(0.0, end, step):
    print n, est(n), act(n)

0.0 0.0844148500014 0.0546723228799
100.0 0.0844148500014 0.0546723228799
200.0 0.100033153116 0.158369479495
300.0 0.100033153116 0.158369479495
400.0 0.193859762397 0.243693107932
500.0 0.193859762397 0.243693107932
600.0 0.267939433838 0.306204136091
700.0 0.267939433838 0.306204136091
800.0 0.315440171774 0.31693989071
900.0 0.315440171774 0.31693989071
1000.0 0.333469221362 0.354009077156
1100.0 0.364829396325 0.365853658537
1200.0 0.364829396325 0.365853658537
1300.0 0.389285714286 0.409090909091
1400.0 0.389285714286 0.409090909091
1500.0 0.417293233083 0.448717948718
1600.0 0.417293233083 0.448717948718
1700.0 0.424242424242 0.382352941176
1800.0 0.424242424242 0.382352941176
1900.0 0.375 0.4
2000.0 0.382978723404 0.333333333333
2100.0 0.382978723404 0.333333333333
2200.0 0.382352941176 0.444444444444
2300.0 0.382352941176 0.444444444444
2400.0 0.478260869565 0.428571428571
2500.0 0.478260869565 0.428571428571
2600.0 0.45 0.571428571429
2700.0 0.45 0.571428571429
2800.0 0.42857

IndexError: index 25 is out of bounds for axis 0 with size 25

In [88]:
ev_est = get_estimator('evalue', 25, mm_ortho, mm_bh)
ev_act = get_actual('evalue', 25, mm_ortho, mm_bh)

In [89]:
mm_bh

,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,sstrand,qstrand
c100000_g1_i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c100000_g1_i2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c100001_g1_i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c100002_g1_i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c100003_g1_i1,ENSMUSP00000046101,53.97,63,26,1,36,216,174,237,1e-17,79,1,-1
c100004_g1_i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c100007_g1_i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c10000_g1_i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c100011_g1_i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c100012_g1_i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
